Part II :

(1) Read in simulated host galaxy data from the catalog built in Part I

(2) Pull out a subset of galaxies around each redshift in [0.8, 1.0, 1.2, 1.5, 1.8, 2.2]

(3) For each redshift subset, identify the host galaxy magnitude that is fainter than 50%, 80% and 95% of the subset in that redshift bin.

(4) Run the Subaru ETC for each of those "percentile marker" galaxies. The ETC outputs a "S/N spectrum", with S/N per pixel vs. wavelength

In [ ]:
%matplotlib inline

In [2]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from matplotlib import pyplot as plt
import snhostspec
from glob import glob

In [3]:
reload(snhostspec)
sim = snhostspec.SnanaSimData()
sim.load_simdata_catalog('wfirst_snhostspec_master.cat')
# pick out the "percentile marker" subset
indexlist = sim.get_host_percentile_indices(zlist=[0.8,1.2,1.5,1.8,2.2])

Initiliazed an empty WfirstSimData object


In [6]:
reload(snhostspec)
sim.load_sed_data()
sim.simulate_host_spectra(indexlist=indexlist)

Loading data for best-fit SEDs from 3DHST fits to observed photometry for all galaxies in all five CANDELS fields...


snhostspec.py:196: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if indexlist is None:


Using Gabe Brammer's EAZY code to generate the best-fit SEDs of the observed galaxies that we have matched up to the SNANA simulation hostgal data.


snhostspec.py:487: RuntimeWarning: divide by zero encountered in double_scalars
  flam_spec = 1. / (1 + z) ** 2
snhostspec.py:488: RuntimeWarning: invalid value encountered in multiply
  tempflux = templf * fnu_factor * flam_spec
/usr/local/anaconda2/envs/astro2/lib/python2.7/site-packages/eazy/igm.py:196: RuntimeWarning: divide by zero encountered in double_scalars
  return a**b


In [7]:
sim.write_catalog("wfirst_snhostspec_master.cat")

Wrote sim data catalog to wfirst_snhostspec_master.cat


In [8]:
reload(snhostspec)
sim.simulate_subaru_snr_curves(indexlist)

Running the PFS ETC for UDS.0768 at z 0.80 with mag 23.33for 1 hrs, sedfile 3DHST/sedsim.output/wfirst_simsed.UDS.0768.dat.
 output: etcout/subaruPFS_SNR_UDS.0768_z0.80_m23.33_1hrs.dat
Finished in 217.2 seconds
Running the PFS ETC for UDS.0768 at z 0.80 with mag 23.33for 5 hrs, sedfile 3DHST/sedsim.output/wfirst_simsed.UDS.0768.dat.
 output: etcout/subaruPFS_SNR_UDS.0768_z0.80_m23.33_5hrs.dat
Finished in 197.7 seconds
Running the PFS ETC for UDS.0768 at z 0.80 with mag 23.33for 10 hrs, sedfile 3DHST/sedsim.output/wfirst_simsed.UDS.0768.dat.
 output: etcout/subaruPFS_SNR_UDS.0768_z0.80_m23.33_10hrs.dat
Finished in 203.6 seconds
Running the PFS ETC for UDS.0768 at z 0.80 with mag 23.33for 40 hrs, sedfile 3DHST/sedsim.output/wfirst_simsed.UDS.0768.dat.
 output: etcout/subaruPFS_SNR_UDS.0768_z0.80_m23.33_40hrs.dat
Finished in 196.9 seconds
Running the PFS ETC for GOODS-N.25261 at z 0.80 with mag 23.84for 1 hrs, sedfile 3DHST/sedsim.output/wfirst_simsed.GOODS-N.25261.dat.
 output: etcout/su

In [ ]:
# BELOW : OLD STUFF needs to be moved or removed.

In [ ]:
fig = plt.figure(figsize=[13,3])
for et, iax  in zip([1, 5, 10,40], [1,2,3,4]):
    ax = fig.add_subplot(1,4,iax)
    ietwin = np.where((zcheck['exptime']==et) & (zcheck['gotz']>0))[0]
    ietfail = np.where((zcheck['exptime']==et) & (zcheck['gotz']<1))[0]
    ax.plot(zcheck['z'][ietwin], zcheck['mag'][ietwin], 'ro', ls=' ')
    ax.plot(zcheck['z'][ietfail], zcheck['mag'][ietfail], 'ks', ls=' ')
ax1 = fig.add_subplot(1,3,1)
ax2 = fig.add_subplot(1,3,2)
ax3 = fig.add_subplot(1,3,3)
ax1.invert_yaxis()
ax2.invert_yaxis()
ax3.invert_yaxis()
ax1.set_title('1hr')
ax2.set_title('5hr')
ax3.set_title('10hr')
ax1.set_ylabel('host gal H band mag')
ax2.set_xlabel('redshift')

In [ ]:
mastercat = wfirst.WfirstMasterHostCatalog()
mastercat.load_all_simdata()
mastercat.write('wfirst_snhostspec_master.cat')

In [ ]:
mastercat2 = wfirst.WfirstMasterHostCatalog()
mastercat2.read('wfirst_snhostspec_master.cat')

In [ ]:
mastercat2.mastercat = ascii.read('wfirst_snhostspec_master.cat', format='commented_header')

In [ ]:
mastercat2.simulate_all_seds()

In [ ]:
reload(wfirst)
sim = wfirst.WfirstSimData('SNANA.SIM.OUTPUT/IMG_2T_4FILT_MD_SLT3_Z08_Ia-01_HEAD.FITS')
sim.load_matchdata('3DHST/3dhst_master.phot.v4.1.cat.FITS')
sim.get_matchlists()
sim.simdata.write("wfirst_snhostgal_sim.dat", format='ascii.commented_header')

Read in the simulated SN data from the SNANA sim data files.
Each SNANA simulation has generated a HEAD.FITS file that contains a binary table with metadata for each simulated SN and host galaxy.  The high-z host galaxy magnitudes have been drawn from distributions that match the CANDELS+CLASH sample -- so there is some selection bias built in, but it will be similar to the selection biases of the WFIRST SN survey (?). 

In [ ]:
reload(wfirst)

In [ ]:
%pwd

In [ ]:
sim.snanadata.read?

In [ ]:
simlist = []
simfilelist_med = glob('SNANA.SIM.OUTPUT/*Z08*HEAD.FITS')
simfilelist_deep = glob('SNANA.SIM.OUTPUT/*Z17*HEAD.FITS')
hostz_med, hostmag_med = np.array([]), np.array([])
for simfile in simfilelist_med:
    sim = wfirst.WfirstSimData(simfile)
    sim.load_matchdata('3DHST/3dhst_master.phot.v4.1.cat.FITS')
    sim.get_matchlists()
    hostz_med = np.append(hostz_med, sim.zsim)
    hostmag_med = np.append(hostmag_med, sim.mag)
    simlist.append(sim)

hostz_deep, hostmag_deep = np.array([]), np.array([])
for simfile in simfilelist_deep:
    sim = wfirst.WfirstSimData(simfile)
    sim.load_matchdata('3DHST/3dhst_master.phot.v4.1.cat.FITS')
    sim.get_matchlists()
    hostz_deep = np.append(hostz_deep, sim.zsim)
    hostmag_deep = np.append(hostmag_deep, sim.mag)
    simlist.append(sim)    

Now for each SNANA sim file, load in the catalog of galaxy SED data from 3DHST and use EAZY to simulate an SED.  The output simulated SEDs are stored in the sub-directory '3dHST/sedsim.output'

In [ ]:
if not os.path.isdir('3DHST/sedsim.output'):
    os.mkdir('3DHST/sedsim.output')
for sim in simlist:
    sim.load_sed_data()
    sim.simulate_seds()

TODO NEXT : run the Subaru ETC on each simulated galaxy spectrum and determine the S/N achieved after 1 hour 5 hour, 10 hour exposures

In [ ]:
# Example of a spectrum plot
eazyspecsim = wfirst.EazySpecSim('3DHST/sedsim.output/wfirst_simsed.AEGIS.0185.dat')
eazyspecsim.plot()

In [ ]:
photdat3d = fits.open('3DHST/3dhst_master.phot.v4.1/3dhst_master.phot.v4.1.cat.FITS')
f160 = photdat3d[1].data['f_F160W']
zspec = photdat3d[1].data['z_spec']
zphot = photdat3d[1].data['z_peak']
zbest = np.where(zspec>0, zspec, zphot)
usephot = photdat3d[1].data['use_phot']

ivalid = np.where(((f160>0) & (zbest>0)) & (usephot==1) )[0]
mH3D = -2.5*np.log10(f160[ivalid])+25
z3D = zbest[ivalid]
plt.plot(z3D, mH3D, 'b.', ls=' ', ms=1, alpha=0.1)
#plt.plot(hostz_med, hostmag_med, 'g.', ls=' ', ms=3, alpha=0.3)
plt.plot(hostz_deep, hostmag_deep, 'r.', ls=' ', ms=3, alpha=0.3)
ax = plt.gca()
xlim = ax.set_xlim(0,2.5)
ylim = ax.set_ylim(28,20)
ax.set_xlabel('redshift')
ax.set_ylabel('host galaxy AB magnitude')